In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import ipyleaflet
import shapely
import pyproj
import contextily
import seaborn as sns
import h3



In [ ]:
#Upload CSV
ruta_csv = "strava_data.csv"
activities = pd.read_csv(ruta_csv)

In [13]:
activities

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_latlng,end_latlng,start_time
0,Afternoon HIIT,15470552955,Workout,0.0,4531,0.000,0.000,0.0,2025-05-16,[],[],17:44:26
1,Evening HIIT,15460331668,Workout,0.0,3774,0.000,0.000,0.0,2025-05-15,[],[],18:24:27
2,Lunch Ride - Innersbachklamm,15449061470,Ride,102228.0,21805,4.688,18.043,545.7,2025-05-14,"[47.810196, 13.038183]","[47.810232, 13.038135]",11:02:05
3,Night Workout,15439339287,Workout,0.0,4233,0.000,0.000,0.0,2025-05-13,[],[],20:38:54
4,Afternoon Workout,15426341743,Workout,0.0,7148,0.000,0.000,0.0,2025-05-12,[],[],17:59:28
...,...,...,...,...,...,...,...,...,...,...,...,...
215,Morning Ride,9917797597,Ride,10296.7,3654,2.818,7.982,344.1,2023-06-10,"[4.297962170094252, -74.78609061799943]","[4.348327768966556, -74.83286884613335]",07:35:28
216,Lunch Walk,9255579410,Walk,3998.6,3009,1.329,1.909,7.7,2023-02-25,"[4.75921, -74.066105]","[4.724326, -74.073518]",12:39:16
217,Arbolito 🌲,8952619873,Ride,23473.4,7171,3.273,10.387,452.4,2023-01-06,"[4.297663, -74.784424]","[4.297661, -74.784449]",06:39:16
218,Ricaurte - Agua de Dios,8948754056,Ride,44084.1,6479,6.804,15.073,375.5,2023-01-05,"[4.295948, -74.786636]","[4.297545, -74.78435]",16:17:54


In [ ]:
# info
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 55 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   resource_state                 220 non-null    int64  
 1   name                           220 non-null    object 
 2   distance                       220 non-null    float64
 3   moving_time                    220 non-null    int64  
 4   elapsed_time                   220 non-null    int64  
 5   total_elevation_gain           220 non-null    float64
 6   type                           220 non-null    object 
 7   sport_type                     220 non-null    object 
 8   id                             220 non-null    int64  
 9   start_date                     220 non-null    object 
 10  start_date_local               220 non-null    object 
 11  timezone                       220 non-null    object 
 12  utc_offset                     220 non-null    flo

In [ ]:
activities.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'id', 'start_date',
       'start_date_local', 'timezone', 'utc_offset', 'location_city',
       'location_state', 'location_country', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'photo_count',
       'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged',
       'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed',
       'has_heartrate', 'average_heartrate', 'max_heartrate',
       'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high',
       'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'workout_type',
       'average_watts', 'device_watts', 'kilojoules'],
      dtype='object')

In [ ]:
#Create new dataframe with only columns I care about
cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
         'average_speed', 'max_speed','total_elevation_gain',
         'start_date_local', 'start_latlng', 'end_latlng'
       ]
activities = activities[cols]

In [ ]:
# format of the date and hour
activities = activities.copy()
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date
activities.head(5)

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_latlng,end_latlng,start_time
0,Afternoon HIIT,15470552955,Workout,0.0,4531,0.000,0.000,0.0,2025-05-16,[],[],17:44:26
1,Evening HIIT,15460331668,Workout,0.0,3774,0.000,0.000,0.0,2025-05-15,[],[],18:24:27
2,Lunch Ride - Innersbachklamm,15449061470,Ride,102228.0,21805,4.688,18.043,545.7,2025-05-14,"[47.810196, 13.038183]","[47.810232, 13.038135]",11:02:05
3,Night Workout,15439339287,Workout,0.0,4233,0.000,0.000,0.0,2025-05-13,[],[],20:38:54
4,Afternoon Workout,15426341743,Workout,0.0,7148,0.000,0.000,0.0,2025-05-12,[],[],17:59:28


In [12]:
# Ver filas donde falla el parseo
print(activities[activities['start_latlng'].isnull() | activities['start_latlng'].isin(['[]', '', 'None'])])


                             name    upload_id            type  distance  \
0                  Afternoon HIIT  15470552955         Workout       0.0   
1                    Evening HIIT  15460331668         Workout       0.0   
3                   Night Workout  15439339287         Workout       0.0   
4               Afternoon Workout  15426341743         Workout       0.0   
5                  Afternoon HIIT  15415012755         Workout       0.0   
..                            ...          ...             ...       ...   
191                Lunch Crossfit  12008937900        Crossfit       0.0   
192       Morning Weight Training  11996027862  WeightTraining       0.0   
193  Afternoon Swim at Uniandes 🐐  11987571574            Swim      33.5   
194              Morning Crossfit  11977681097        Crossfit       0.0   
196  Afternoon Swim at Uniandes 🐐  11942623045            Swim    1100.0   

     moving_time  average_speed  max_speed  total_elevation_gain  \
0           4531   

In [14]:
def extract_lat_lon(latlng_str):
    try:
        lat, lon = latlng_str.strip('[]').split(',')
        return float(lat), float(lon)
    except:
        return None, None


In [15]:
activities['start_lat'], activities['start_lon'] = zip(*activities['start_latlng'].map(extract_lat_lon))


In [16]:
activities

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_latlng,end_latlng,start_time,start_lat,start_lon
0,Afternoon HIIT,15470552955,Workout,0.0,4531,0.000,0.000,0.0,2025-05-16,[],[],17:44:26,NaN,NaN
1,Evening HIIT,15460331668,Workout,0.0,3774,0.000,0.000,0.0,2025-05-15,[],[],18:24:27,NaN,NaN
2,Lunch Ride - Innersbachklamm,15449061470,Ride,102228.0,21805,4.688,18.043,545.7,2025-05-14,"[47.810196, 13.038183]","[47.810232, 13.038135]",11:02:05,47.810196,13.038183
3,Night Workout,15439339287,Workout,0.0,4233,0.000,0.000,0.0,2025-05-13,[],[],20:38:54,NaN,NaN
4,Afternoon Workout,15426341743,Workout,0.0,7148,0.000,0.000,0.0,2025-05-12,[],[],17:59:28,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Morning Ride,9917797597,Ride,10296.7,3654,2.818,7.982,344.1,2023-06-10,"[4.297962170094252, -74.78609061799943]","[4.348327768966556, -74.83286884613335]",07:35:28,4.297962,-74.786091
216,Lunch Walk,9255579410,Walk,3998.6,3009,1.329,1.909,7.7,2023-02-25,"[4.75921, -74.066105]","[4.724326, -74.073518]",12:39:16,4.759210,-74.066105
217,Arbolito 🌲,8952619873,Ride,23473.4,7171,3.273,10.387,452.4,2023-01-06,"[4.297663, -74.784424]","[4.297661, -74.784449]",06:39:16,4.297663,-74.784424
218,Ricaurte - Agua de Dios,8948754056,Ride,44084.1,6479,6.804,15.073,375.5,2023-01-05,"[4.295948, -74.786636]","[4.297545, -74.78435]",16:17:54,4.295948,-74.786636


In [20]:
from shapely.geometry import Point

# Create Point geometry from latitude and longitude
geometry = [Point(xy) for xy in zip(activities['start_lon'], activities['start_lat'])]

# Create a GeoDataFrame with WGS84 coordinate system
gdf = gpd.GeoDataFrame(activities, geometry=geometry, crs="EPSG:4326")

#Delete nan
gdf = gdf.dropna(subset=['start_lat', 'start_lon'])


In [28]:
# Create the folium map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add each activity as a CircleMarker
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row['start_lat'], row['start_lon']],
        radius=5,
        popup=f"{row['name']} ({row['type']})",
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Save the map to an HTML file
m.save('strava_start_points.html')



In [29]:
m